In [1]:
import numpy as np
import torch

from torch.utils.data import Dataset

import torch.nn.functional as F
import torch.nn as nn

import matplotlib.pyplot as plt

import myModelinTF

from torch.optim.lr_scheduler import StepLR
import pandas as pd
import tensorflow as tf
from torch.autograd import Variable 

In [2]:
def cumcount(arr):
    result = []
    count = 0
    curr_individual = arr[0]
    result.append(count)
    for value in arr[1:]:        
        if value == curr_individual:
            count += 1
            result.append(count)         
        else :
            count = 0
            result.append(count)
            curr_individual = value
    return result

# input_array = np.array([1, 1, 1, 2, 2, 1, 1, 1, 1])
# output_array = cumcount(input_array)
# print(output_array)

In [3]:
# A = np.array([[1,2],[3,4],[1,2],[3,4]])

# print(np.pad(A, ((0,5-len(A)),(0,0)), 'constant'))

In [56]:
class SeriesDataset(Dataset):

    def __init__(self, file):
        self.demo = np.load(file, allow_pickle=True)

    #len(usd)
    def __len__(self):
        return len(self.demo)

    # a_list[1] --> a__list.__getitem__(1)
    def __getitem__(self, index):
        run =self.demo[index]

        states = np.array(run[0]).T
        states_init1 = states[1,:]
        actions = np.array(run[1])[:, None]

        df_actions_counts = pd.DataFrame(cumcount(np.squeeze(actions)),columns =['cum_counts'])
        # the last value is padded with Nan, so putting 1000 as value, to not fetch with the 0
        df_actions_counts['lead_counts'] = df_actions_counts['cum_counts'].shift(-2).replace(np.nan, 1000)
        df_actions_counts['actions'] = np.squeeze(actions).tolist()
        df_actions_counts_at_0 = df_actions_counts.loc[df_actions_counts['lead_counts'] == 0]
        df_actions_counts['cum_counts'] = df_actions_counts['cum_counts']+1
        actions_counts_array = df_actions_counts_at_0[['actions', 'cum_counts']]
        actions_counts_array_padded = np.pad(actions_counts_array, ((0,500-len(actions_counts_array)),(0,0)), 'constant').astype('int32') 

        states_init = np.zeros((len(actions_counts_array_padded), 10)) + states_init1
        return states_init, actions_counts_array_padded

In [5]:
def permute_them(x):
  return torch.permute(x, (1, 0, 2))

In [6]:
def collate_fn(batch):

    tensors, targets = [], []

    for states_init, actions in batch:
        tensors += [torch.tensor(states_init).type(torch.FloatTensor)]
        targets += [torch.tensor(actions).type(torch.LongTensor)]

    targets = torch.stack(targets)

    return tensors, targets

In [48]:
class PG(nn.Module):
    def __init__(self, state_shape, n_actions):
        super().__init__()
        self.state_shape = state_shape
        self.n_actions = n_actions

        # Policy takes input initial conditions and outputs planning route, open-loop control
        self.model = nn.Sequential(
            nn.LSTM(input_size = 10, hidden_size = 5, num_layers = 2, dropout = 0.1,batch_first = True),
            
        )

    def forward(self, states):
        #states = torch.FloatTensor(states)
        logits, (h_T, c_T)  = self.model(states)
        return logits
    
    def predict_probs(self, states):
        states = torch.FloatTensor(states)
        logits, (h_T, c_T) = self.model(states)
        probs = F.softmax(logits, dim = -1).detach().numpy()
        return probs
    
    # Run agent in environment to create sample trajectories by generator
    # The environment model is a seq2seq model
    def generate_session(self, sysmodel, t_max=1000):
        norm_params = pd.read_csv('D:\\units\\thesis\\thesis\\train_reward_inverse_rl_w_sys_model\\norm_params.csv', header = None).to_numpy().T

        states, actions, traj_probs, rewards = [], [], [], []
        states_init = np.zeros((500,1, 10)) + np.random.randint(20, 34, size=(10)) 
        actions_probs_policy = self.predict_probs(states_init)
        actions = []
        actions_probs_policy = np.squeeze(actions_probs_policy)     
        for prob in actions_probs_policy:                  
            actions.append(np.random.choice(self.n_actions,  p = prob))
 
        # Apply dict to go from 1,2,3,4 as action to [0,2400,0] [0,2400,1000]  [2300,0,0]  [2300,0,1000] 
        my_dict = {0:[0,2400,0], 1:[0,2400,1000], 2:[2300,0,0], 3:[2300,0,1000]}
        actions_array = np.zeros((500,3))  
        actions_array = map(my_dict.get, actions)

        actions_array = np.array(list(actions_array))
        actions_array_sysmodel = np.concatenate([np.arange(actions_array.shape[0])[:,None]+1,actions_array], axis=1) 
        actions_array_sysmodel = tf.expand_dims(actions_array_sysmodel, axis = 0, name=None)

        states = sysmodel(actions_array_sysmodel)
        states = states*norm_params[:,0]+norm_params[:,1]
        
        return states, actions, actions_probs_policy

In [27]:
def train(model, tepoch):

    model.train()
    losses = []
    batch_idx = 0

    for data, target in train_loader:
        correct = 0
        data = torch.stack(data)
        
        output = model(data)
        target0 = to_one_hot(target[:,:,0], 4).float()
        
        output0 = torch.flatten(output[:,:,0:4], start_dim = 0, end_dim = 1)
        loss = criterion2(torch.squeeze(output[:,:,4]), torch.squeeze(target[:,:,1]).float()) + criterion1(output0, target0) 

        output_np = output.detach().numpy().tolist()
        target_np = target.detach().numpy().tolist()  
        for (output, target) in zip(output_np, target_np):        
            if output.index(max(output)) == target.index(max(target)):
                correct +=1
        
        accuracy = correct / (500 * BATCH_SIZE)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print training stats
        print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}\tAccuracy of batch: {accuracy:.6f}")
        # record loss
        
        losses.append(loss.item())

        batch_idx += 1

In [ ]:
def test(model, epoch):
    model.eval()
    correct = 0

    for data, target in test_loader:
        data = torch.stack(data)

        output = model(data)
        target0 = to_one_hot(target[:,:,0], 4)
        
        # action to take
        output0 = torch.flatten(output[:,:,0:4], start_dim = 0, end_dim = 1)
        # aount
        output1 = torch.squeeze(output[:,:,4])

        output_np = output0.detach().numpy().tolist()
        target_np = target0.detach().numpy().tolist()
        correct = 0
        for (output3, target) in zip(output_np, target_np):           
            if output3.index(max(output3)) == target.index(max(target)):
                correct +=1

        accuracy = correct / (500 * BATCH_SIZE)

    print(f"\nTest Epoch: {epoch}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\tAccuracy: {accuracy}\n")
    for out in output:
        probs = nn.functional.softmax(out[:,0:4], -1) # get estimated actions for states
        actions = []
        A = [0,1,2,3]
        probs = np.squeeze(probs).detach().numpy()
        
        for prob in probs:                  
            actions.append(A[np.argmax(prob)])

        print(actions)
        print(out[:,4])

        
        plt.subplot(2, 1, 1)
        plt.title(f"Example Initial state")
        plt.plot(np.squeeze(data).detach().numpy())
        plt.grid()

        # plt.subplot(2, 1, 2)
        # plt.title(f"Actions")
        # plt.plot(actions)
        plt.subplot(2,1,2)

        plt.stairs(actions, linewidth=2.5)

        plt.xlim=(0, 8)
        plt.xticks=out[:,4]
        plt.ylim=(0, 3)
        plt.yticks=[0,1,2,3]
        plt.grid()

        # plt.show()
        plt.savefig('Train LSTM_action sequence.png')
        plt.close()

In [21]:
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()

def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)

def compute_error(x, y):
    Ps = torch.norm(x)
    Pn = torch.norm(x-y)
    return 20*torch.log10(Ps/Pn)

def get_expected_rewards(rewards,gamma=0.9):
    G = np.zeros_like(rewards, dtype=float)
    G[-1] = rewards[-1]
    for idx in range(-2, -len(rewards)-1, -1):
        G[idx] = rewards[idx] + gamma * G[idx+1]
    return G

def get_rewards(states, actions):
    rewards = np.zeros((500,1))

    states = states.numpy()
    states = states.squeeze()

    max_hotspot =  np.max(states)
    if max_hotspot < 300:
        rewards[-1] = 50

    if np.where(states[:,1] == 230)[0] < 405 & np.where(states[:,1] == 230)[0] > 340:
        rewards[-1] = rewards[-1] + 40
        
    for idx in range(np.shape(states)[0]):
        temps = np.array(states[idx])
        if temps.all() > 300:
            rewards[idx] = rewards[idx] -30
        
        subset = actions[idx-30:]
        if np.sum(np.diff(subset)) == 0:
            rewards[idx] = rewards[idx] + 20

        subset = temps[idx-20:]
        if np.sum(np.diff(subset)) < 10:
            rewards[idx] = rewards[idx] + 20

        if actions[idx] == 1 | actions[idx] == 3:
            rewards[idx] = rewards[idx] -10


    return rewards

def to_one_hot(y_tensor, ndims):
    y_tensor = y_tensor.type(torch.LongTensor).view(-1, 1)
    y_one_hot = torch.zeros(
        y_tensor.size()[0], ndims).scatter_(1, y_tensor, 1)
    return y_one_hot

In [11]:
# CONVERTS TRAJ LIST TO STEP LIST
def preprocess_traj(traj_list, step_list):
    step_list = step_list.tolist()
    for traj in traj_list:
        states = np.array(traj[0])
        probs = np.ones((500,4))
        states = np.transpose(traj[0])
        actions = np.array(traj[1])[:, None]

        x = np.concatenate((states, actions, probs), axis=1) 
        step_list.extend(x)
    return np.array(step_list)


In [ ]:
# DEVICE
DEVICE = 'cpu'

# ENV SETUP
sysmodel = myModelinTF.load_model()
n_actions = 4
state = np.zeros((500,10)) + 25
state_shape = state.shape

# INITILIZING POLICY AND REWARD FUNCTION
policy = PG(state_shape, n_actions)
policy.to(DEVICE)
norm_params = pd.read_csv('D:\\units\\thesis\\thesis\\train_reward_inverse_rl_w_sys_model\\norm_params.csv', header = None).to_numpy().T



###### PRE-TRAIN WITH EXISTING DATA THE POLICY
N_EPOCH = 50
optimizer= torch.optim.Adam(policy.parameters(), 1e-2, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1) 
criterion1 = nn.CrossEntropyLoss()
criterion2 = nn.MSELoss()


# LOADING EXPERT/DEMO SAMPLES
DATASETNPY_TRAIN = "dataset_train.npy"
DATASETNPY_TEST = "dataset_test.npy"

train_set = SeriesDataset(DATASETNPY_TRAIN)
test_set = SeriesDataset(DATASETNPY_TEST)

#print("Train set size: " + str(len(train_set)))
BATCH_SIZE = 1

train_loader = torch.utils.data.DataLoader(
        train_set,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=collate_fn,
    )
test_loader = torch.utils.data.DataLoader(
        test_set,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=collate_fn,
    )

for epoch in range(1, N_EPOCH + 1):
    train(policy, epoch)
    test(policy, epoch)
    scheduler.step()

##### TUNE MODEL WITH NEW LOSS

mean_rewards = []
mean_loss = []
mean_loss_rew = []
EPISODES_TO_PLAY = 10
sample_trajs = []

return_list, sum_of_loss_list = [], []

for child in policy.children():
    print(child)
    k=1
    for param in child.parameters():
            if k % 4 == 0:
                param.requires_grad = False
            k+=1

optimizer_policy = torch.optim.Adam(filter(lambda p: p.requires_grad, policy.parameters()), 1e-1, weight_decay=1e-2)
for i in range(EPISODES_TO_PLAY):

    states_init = np.zeros((1,500, 10)) + np.random.randint(20, 34, size=(10))
    states_init = torch.FloatTensor(states_init)
    output = policy(states_init) # forward pass
    logits = output[:,:,0:4]
    durations = output[:,:,4].detach().numpy()

    durations = np.rint(durations) 
    probs = nn.functional.softmax(logits, -1) # get estimated actions for states
    #log_probs = nn.functional.log_softmax(logits, -1)
    
    actions_probs_policy = np.squeeze(probs).detach().numpy()
    # take actions only until first appearnce of duration 0
    dur0_idx = np.where(durations == 0)
    actions_probs_policy = actions_probs_policy[:dur0_idx[1][0]] 

    durations = np.transpose(durations)
    durations = durations[:dur0_idx[1][0]]   
    
    actions_ind = []
    actions = []
    for prob in actions_probs_policy:                  
        actions_ind.append(np.random.choice(4,  p = prob))

    # actions was an array of individual type of actions to take, needs to be expanded with durations
    actions_ind = np.array(actions_ind)
    actions = np.repeat(actions_ind,durations[:,0].astype(int))
    # Apply dict to go from 1,2,3,4 as action to [0,2400,0] [0,2400,1000]  [2300,0,0]  [2300,0,1000] 
    my_dict = {0:[0,2400,0], 1:[0,2400,1000], 2:[2300,0,0], 3:[2300,0,1000]}
    actions_array = np.zeros((500,3))  
    actions_array = map(my_dict.get, actions[1:500])

    actions_array = np.array(list(actions_array))
    actions_array_sysmodel = np.concatenate([np.arange(actions_array.shape[0])[:,None]+1,actions_array], axis=1) 
    actions_array_sysmodel = tf.expand_dims(actions_array_sysmodel, axis = 0, name=None)

    states = sysmodel(actions_array_sysmodel)

    states = states*norm_params[:,0]+norm_params[:,1]

    # Recursively get expected discounted rewards (rewards given by the current cost function)
    rewards = get_rewards(states, actions)
    cumulative_returns_np = np.array(get_expected_rewards(rewards, 0.9))
    cumulative_returns = torch.tensor(cumulative_returns_np, dtype=torch.float32)

    actions_tensor = torch.from_numpy(np.array(actions))
    log_probs_for_actions = torch.sum(
        probs[:, 0:dur0_idx[1][0]] * to_one_hot(actions_tensor, 4), dim=1)

    #entropy = -torch.mean(torch.sum(probs_samp*log_probs_samp), dim = -1 )
    #loss = -torch.mean(log_probs_for_actions*cumulative_returns - entropy*1e-2) # loss for the policy (isnt it the cost function output?)
    #average reward baseline
    cumulative_returns = (cumulative_returns - torch.mean(cumulative_returns))
    #print(cumulative_returns)
    #whitening baseline
    #cumulative_returns = (cumulative_returns - torch.mean(cumulative_returns))/ (torch.std(cumulative_returns))
    loss_policy = -log_probs_for_actions*cumulative_returns/1e-3
    # UPDATING THE POLICY NETWORK
    optimizer_policy.zero_grad()
    loss_policy.sum().backward()
    torch.nn.utils.clip_grad_norm_(policy.parameters(), 0.01)
    optimizer_policy.step()

    #returns = sum(rewards)
    sum_of_loss = loss_policy.sum().detach().numpy()
    print(f"\nLoss: {sum_of_loss}\n")
    #print(f"\nCumulative return: {cumulative_returns}\n")
    #return_list.append(returns)
    sum_of_loss_list.append(sum_of_loss)
    states_plot = np.squeeze(states)

    plt.subplot(2, 2, 1)
    plt.title(f"Loss per {EPISODES_TO_PLAY} episodes")
    plt.plot(sum_of_loss_list)
    plt.grid()

    plt.subplot(2, 2, 2)
    plt.title(f"Rewards")
    plt.plot(rewards)
    plt.grid()

    plt.subplot(2, 2, 3)
    plt.title(f"Activations")
    plt.plot(actions[450:500])
    plt.grid()

    plt.subplot(2, 2, 4)
    plt.title(f"Temperatures")
    plt.plot(states_plot[0:50,:])
    plt.grid()

    # plt.show()
    plt.savefig('GCL_learning_curve.png')
    plt.close()